#### MAIN THINGS TO DO

- Clean the data (remove unnamed columns)
- Currently working with 2015-16 data
- Current plan: 1 model for each season
- Split the data into training and testing
- Create and train model (linear regression)
- Test

#### OTHER IMPORTANT STUFF

- Upload code to GitHub


In [167]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [168]:
import seaborn as sns

In [169]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
# How to filter

#filter1 = (raw_data['Entity'] == "United States")
#data = raw_data[filter1]
#data

In [247]:
# load the data

nikola_15_original = pd.read_csv("./data/data_nikola_2015-16.csv")
nikola_16_original = pd.read_csv("./data/data_nikola_2016-17.csv")

In [248]:
nikola_15_original.head()

,Rk,G,Date,Age,Tm,Unnamed: 5,Opp,Unnamed: 7,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,1,1.0,2015-10-28,20-251,DEN,@,HOU,W (+20),0,3:55,...,0,0,0,0,0,1,0,2,0.7,+1
1,2,2.0,2015-10-30,20-253,DEN,NaN,MIN,L (-17),0,18:09,...,6,9,1,0,0,2,3,10,7.8,+10
2,3,3.0,2015-11-01,20-255,DEN,@,OKC,L (-24),0,24:26,...,1,4,0,0,0,1,2,8,3.9,-2
3,4,NaN,2015-11-03,20-257,DEN,@,LAL,W (+11),Did Not Dress,Did Not Dress,...,Did Not Dress,Did Not Dress,Did Not Dress,Did Not Dress,Did Not Dress,Did Not Dress,Did Not Dress,Did Not Dress,Did Not Dress,Did Not Dress
4,5,4.0,2015-11-05,20-259,DEN,NaN,UTA,L (-12),0,17:44,...,2,3,0,0,1,1,1,5,2.8,-6


In [210]:
nikola_16_original.head()

,Rk,G,Date,Age,Tm,Unnamed: 5,Opp,Unnamed: 7,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,1,1.0,2016-10-26,21-250,DEN,@,NOP,W (+5),1,21:08,...,6,7,2,0,0,4,4,5,0.6,+5
1,2,2.0,2016-10-29,21-253,DEN,NaN,POR,L (-2),1,34:06,...,11,17,2,4,1,1,5,23,23.2,-1
2,3,3.0,2016-10-31,21-255,DEN,@,TOR,L (-3),1,22:22,...,2,4,1,0,0,4,4,12,5.2,-9
3,4,4.0,2016-11-03,21-258,DEN,@,MIN,W (+3),1,14:26,...,2,3,5,0,0,1,3,8,8.9,+7
4,5,5.0,2016-11-05,21-260,DEN,@,DET,L (-17),1,22:42,...,4,6,0,1,1,1,3,6,3.0,-14


In [211]:
nikola_15_original.columns

Index(['Rk', 'G', 'Date', 'Age', 'Tm', 'Unnamed: 5', 'Opp', 'Unnamed: 7', 'GS',
       'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-'],
      dtype='object')

In [249]:
nikola_15_original.rename(columns = {'Unnamed: 5' : 'Loc'}, inplace = True)

nikola_15 = nikola_15_original.drop(labels = ['Unnamed: 7'], axis = 1)
nikola_15.dropna(subset=['G'], inplace = True)

nikola_15['Loc'] = nikola_15['Loc'].astype('str')

def loc_change(x):
    if x == '@':
        return 'Away'
    elif x == "nan":
        return 'Home'

nikola_15['Loc'] = nikola_15['Loc'].apply(loc_change)

In [215]:
nikola_15.columns

Index(['Rk', 'G', 'Date', 'Age', 'Tm', 'Loc', 'Opp', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-'],
      dtype='object')

In [254]:
nikola_15

,Rk,G,Date,Age,Tm,Loc,Opp,GS,MP,FG,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,1,1.0,2015-10-28,20-251,DEN,Away,HOU,0,3:55,1,...,0,0,0,0,0,1,0,2,0.7,+1
1,2,2.0,2015-10-30,20-253,DEN,Home,MIN,0,18:09,5,...,6,9,1,0,0,2,3,10,7.8,+10
2,3,3.0,2015-11-01,20-255,DEN,Away,OKC,0,24:26,4,...,1,4,0,0,0,1,2,8,3.9,-2
4,5,4.0,2015-11-05,20-259,DEN,Home,UTA,0,17:44,1,...,2,3,0,0,1,1,1,5,2.8,-6
5,6,5.0,2015-11-06,20-260,DEN,Away,GSW,0,17:00,2,...,2,4,1,1,1,1,1,4,4.2,-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,78,76.0,2016-04-02,21-043,DEN,Home,SAC,1,32:14,2,...,9,13,5,1,2,0,2,6,13.2,-1
78,79,77.0,2016-04-05,21-046,DEN,Home,OKC,1,19:03,3,...,2,2,2,0,2,0,2,7,7.3,-9
79,80,78.0,2016-04-08,21-049,DEN,Home,SAS,1,32:13,3,...,12,15,4,1,1,0,2,8,12.3,+24
80,81,79.0,2016-04-10,21-051,DEN,Home,UTA,1,36:02,7,...,8,11,4,2,1,1,4,19,22.5,-16


In [255]:
nikola_15.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80 entries, 0 to 81
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Rk      80 non-null     int64  
 1   G       80 non-null     float64
 2   Date    80 non-null     object 
 3   Age     80 non-null     object 
 4   Tm      80 non-null     object 
 5   Loc     80 non-null     object 
 6   Opp     80 non-null     object 
 7   GS      80 non-null     object 
 8   MP      80 non-null     object 
 9   FG      80 non-null     object 
 10  FGA     80 non-null     object 
 11  FG%     76 non-null     object 
 12  3P      80 non-null     object 
 13  3PA     80 non-null     object 
 14  3P%     53 non-null     object 
 15  FT      80 non-null     object 
 16  FTA     80 non-null     object 
 17  FT%     53 non-null     object 
 18  ORB     80 non-null     object 
 19  DRB     80 non-null     object 
 20  TRB     80 non-null     object 
 21  AST     80 non-null     object 
 22  STL     8

In [256]:
# number of null values per column
nikola_15.isnull().sum()


Rk       0
G        0
Date     0
Age      0
Tm       0
Loc      0
Opp      0
GS       0
MP       0
FG       0
FGA      0
FG%      4
3P       0
3PA      0
3P%     27
FT       0
FTA      0
FT%     27
ORB      0
DRB      0
TRB      0
AST      0
STL      0
BLK      0
TOV      0
PF       0
PTS      0
GmSc     0
+/-      0
dtype: int64

In [269]:
# New dataframe w/o Tm, FG%, 3P%, and FT% --> we had null values so we removed those

nikola_15_data = nikola_15.drop(labels=['Rk', 'Date', 'G', 'Tm', 'FG%', '3P%', 'FT%'], axis=1)
nikola_15_data.columns

Index(['Age', 'Loc', 'Opp', 'GS', 'MP', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc',
       '+/-'],
      dtype='object')

**FIXING DATA TYPES**

In [270]:
# we need to change the datatypes here

nikola_15_data.dtypes

Age     object
Loc     object
Opp     object
GS      object
MP      object
FG      object
FGA     object
3P      object
3PA     object
FT      object
FTA     object
ORB     object
DRB     object
TRB     object
AST     object
STL     object
BLK     object
TOV     object
PF      object
PTS     object
GmSc    object
+/-     object
dtype: object

In [271]:
def convert_mp(mp_time): # WORKS!

    vals = mp_time.split(':')

    seconds = float(vals[1])

    seconds_in_minutes = seconds / 60.0

    return round(float(vals[0]) + seconds_in_minutes, 2)

def convert_age(age): # WORKS!

    vals = age.split('-')

    years = int(vals[0])

    return years


In [273]:
# run this once only

# datetime

# nikola_15_data['Date'] = pd.to_datetime(nikola_15_data['Date'])


# columns to change to int values

cols = ['FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 
        'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-']

nikola_15_data[cols] = nikola_15_data[cols].astype(int)


# columns to change to float values

nikola_15_data['GmSc'] = nikola_15_data['GmSc'].astype(float)


# apply both functions -- changing MP and Age to floats/ints

nikola_15_data['MP'] = nikola_15_data['MP'].apply(convert_mp)
nikola_15_data['Age'] = nikola_15_data['Age'].apply(convert_age)

# columns to change to factors

cols_factor = ['Loc', 'Opp', 'GS']

nikola_15_data[cols_factor] = nikola_15_data[cols_factor].astype('category')

In [285]:
nikola_15_data

,Age,Loc,Opp,GS,MP,FG,FGA,3P,3PA,FT,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,20,Away,HOU,0,3.92,1,1,0,0,0,...,0,0,0,0,0,1,0,2,0.7,1
1,20,Home,MIN,0,18.15,5,8,0,0,0,...,6,9,1,0,0,2,3,10,7.8,10
2,20,Away,OKC,0,24.43,4,9,0,0,0,...,1,4,0,0,0,1,2,8,3.9,-2
4,20,Home,UTA,0,17.73,1,4,0,0,3,...,2,3,0,0,1,1,1,5,2.8,-6
5,20,Away,GSW,0,17.00,2,4,0,0,0,...,2,4,1,1,1,1,1,4,4.2,-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,21,Home,SAC,1,32.23,2,6,0,1,2,...,9,13,5,1,2,0,2,6,13.2,-1
78,21,Home,OKC,1,19.05,3,5,1,3,0,...,2,2,2,0,2,0,2,7,7.3,-9
79,21,Home,SAS,1,32.22,3,9,0,2,2,...,12,15,4,1,1,0,2,8,12.3,24
80,21,Home,UTA,1,36.03,7,9,0,2,5,...,8,11,4,2,1,1,4,19,22.5,-16


In [274]:
nikola_15_data.head()

,Age,Loc,Opp,GS,MP,FG,FGA,3P,3PA,FT,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,20,Away,HOU,0,3.92,1,1,0,0,0,...,0,0,0,0,0,1,0,2,0.7,1
1,20,Home,MIN,0,18.15,5,8,0,0,0,...,6,9,1,0,0,2,3,10,7.8,10
2,20,Away,OKC,0,24.43,4,9,0,0,0,...,1,4,0,0,0,1,2,8,3.9,-2
4,20,Home,UTA,0,17.73,1,4,0,0,3,...,2,3,0,0,1,1,1,5,2.8,-6
5,20,Away,GSW,0,17.00,2,4,0,0,0,...,2,4,1,1,1,1,1,4,4.2,-5


In [275]:
# check datatypes

nikola_15_data.dtypes

Age        int64
Loc     category
Opp     category
GS      category
MP       float64
FG         int64
FGA        int64
3P         int64
3PA        int64
FT         int64
FTA        int64
ORB        int64
DRB        int64
TRB        int64
AST        int64
STL        int64
BLK        int64
TOV        int64
PF         int64
PTS        int64
GmSc     float64
+/-        int64
dtype: object

# Modeling

In [288]:
numerical_features = ['FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 
        'AST', 'STL', 'BLK', 'TOV', 'PF', '+/-', 'GmSc', 'MP', 'Age']

categorical_features = ['Loc', 'Opp', 'GS']

In [290]:
# get input features and output feature, split data into training and testing
output = nikola_15_data[['PTS']].copy()
features = nikola_15_data.drop('PTS', axis=1)


X_train, X_test, y_train, y_test = train_test_split(features, output, test_size=0.2, random_state = 50)

len(X_train), len(X_test), len(y_train), len(y_test)

(65, 17, 65, 17)

In [ ]:
# current problems: category columns
# for Loc column -> make it binary? 0 = home, 1 = away?
# for opposite team column -> use onehotencoder? = where HOU can be 001, MIN can be 010, etc -> small problem = dont really know how to do it
# map list of team to binary number? then just apply function?

In [301]:
team_names = list(set(nikola_15_original['Opp'].values))

team_dict = dict()

index = 0

for name in team_names:
    team_dict[name] = index
    index += 1

print(team_dict)

{'ATL': 0, 'OKC': 1, 'DET': 2, 'HOU': 3, 'CHI': 4, 'BOS': 5, 'BRK': 6, 'MEM': 7, 'WAS': 8, 'GSW': 9, 'DAL': 10, 'MIA': 11, 'UTA': 12, 'ORL': 13, 'PHO': 14, 'LAL': 15, 'SAS': 16, 'IND': 17, 'MIL': 18, 'TOR': 19, 'MIN': 20, 'NOP': 21, 'SAC': 22, 'LAC': 23, 'PHI': 24, 'NYK': 25, 'CHO': 26, 'CLE': 27, 'POR': 28}


In [ ]:
# next step -> convert values in dictionary to binary